### Import Libs

In [48]:
import matplotlib.pyplot as plt
import numpy as np
import random

### Create Individual

In [37]:
def create_individual():
    # TODO: create a random chromosome
    arr = np.array(range(1,9,1))
    np.random.shuffle(arr)
    
    return arr.tolist()

### Generate Population

In [38]:
def generate_population(population_size:int = 20):
    population = []

    for _ in range(population_size):
        new_individual = create_individual()
        population.append(new_individual)
    
    return population

In [39]:
generate_population()

[[6, 4, 7, 1, 3, 2, 8, 5],
 [3, 2, 6, 8, 4, 1, 7, 5],
 [4, 6, 5, 1, 3, 8, 2, 7],
 [3, 8, 2, 5, 7, 1, 6, 4],
 [5, 4, 8, 1, 2, 6, 7, 3],
 [4, 2, 1, 5, 6, 3, 8, 7],
 [8, 4, 3, 6, 7, 2, 1, 5],
 [5, 4, 6, 8, 3, 1, 2, 7],
 [6, 1, 3, 8, 7, 4, 5, 2],
 [8, 7, 5, 6, 4, 1, 3, 2],
 [7, 6, 8, 4, 1, 5, 3, 2],
 [6, 2, 1, 7, 8, 5, 3, 4],
 [2, 7, 1, 5, 3, 4, 6, 8],
 [5, 1, 6, 8, 3, 2, 4, 7],
 [6, 2, 4, 5, 8, 7, 3, 1],
 [8, 4, 6, 5, 3, 2, 7, 1],
 [7, 6, 1, 2, 3, 4, 8, 5],
 [6, 8, 1, 2, 3, 4, 7, 5],
 [1, 4, 6, 8, 3, 2, 7, 5],
 [1, 8, 5, 4, 7, 6, 2, 3]]

### Parent Selection

In [47]:
def SUS(population: dict, population_size:int = 20):
    probability_ruler = []
    population_keys = list(population.keys())
    
    selected_list = []
    
    for i in range(len(population)):
        if i == 0:
            probability_ruler.append(population[population_keys[i]])
            continue
        probability_ruler.append(population[population_keys[i]] + population[population_keys[i-1]])
    
    rand_prob = random.random() / population_size
    
    index = 0
    for i in range(population_size):
        current_prob = rand_prob + i / population_size
        if current_prob < probability_ruler[index]:
            selected_list.append(population_keys[index])
        else:
            index += 1
            for j in range(index, len(population_keys)):
                if current_prob < probability_ruler[j]:
                    index = j
                    selected_list.append(population_keys[index])
                    break
            
    return selected_list

In [41]:
def parent_selection(population: dict ,population_size:int = 20):
    #TODO: choose the pairs of parents
    sum_population = sum(population.values())
    prob_population = {}
    for p in population.keys():
        prob_population[p] = population[p] / sum_population
    
    parents = SUS(prob_population,population_size)
            
    return parents

### Next Generation Selection

In [ ]:
def next_generation_selection(children_population):
    next_generation = []
    #TODO: choose the next_generation
            
    return next_generation

### Crossover

In [ ]:
def crossover(parents):
    offsprings = []
    # TODO: implement you crossover algorithm
    parents = np.array(parents)
    np.random.shuffle(parents)
    queen_number = len(parents[0].split(" "))

    for i in range(int(len(parents) / 2)):
        if random.random() > 0.8:
            continue

        crossover_point = random.randint(0, queen_number - 1)

        parent_1 = parents[2 * i].split(" ")
        parent_2 = parents[2 * i + 1].split(" ")

        offspring_1 = parent_1[:crossover_point]
        offspring_2 = parent_2[:crossover_point]

        index = crossover_point
        for j in range(queen_number - crossover_point):
            if parent_2[index] not in offspring_1:
                offspring_1.append(parent_2[index])
                index = (index + 1) % queen_number
                continue

            while parent_2[index] in offspring_1:
                index = (index + 1) % queen_number

            offspring_1.append(parent_2[index])

        index = crossover_point
        for j in range(queen_number - crossover_point):
            if parent_1[index] not in offspring_2:
                offspring_2.append(parent_1[index])
                index = (index + 1) % queen_number
                continue

            while parent_1[index] in offspring_2:
                index = (index + 1) % queen_number

            offspring_2.append(parent_1[index])

        offsprings.append(offspring_1)
        offsprings.append(offspring_2)

    return offsprings

### Mutation

In [ ]:
def mutate(seq):
    #TODO: implement mutate
    
    return seq

### Score Function

In [42]:
def fitness_score(seq):
    fit_score = 0
    #TODO: calculate fitness score
    status_queen = np.ones(len(seq))
    for current_row in range(len(seq)):
        flag = 1
        for other_row in range(current_row+1,len(seq)):
            if seq[current_row] == seq[other_row]:
                print(current_row, other_row)
                status_queen[current_row] = 0
                status_queen[other_row] = 0
                flag = 0
                continue
            if abs(seq[current_row] - seq[other_row]) == abs(current_row - other_row):
                print(current_row, other_row)
                status_queen[current_row] = 0
                status_queen[other_row] = 0
                flag = 0
                continue
        
        if flag==1 and status_queen[current_row] == 1: 
            print(flag , current_row)
            fit_score += 1
    return fit_score

In [43]:
fitness_score([5, 4, 7, 6, 2, 3, 1, 8])

0 1
0 2
1 3
2 3
4 5
1 6
1 7


2

## Calculate Population Fitness

In [44]:
def population_fitness(population):
    population_fitness = 0
    individuals_fitnesses:dict = {}
    for chromosome in population:
        fitness_sc = fitness_score(chromosome)
        population_fitness += fitness_sc
        ch= ' '.join(str(x) for x in chromosome)
        # print(ch)
        individuals_fitnesses[ch] = fitness_sc
    return individuals_fitnesses ,population_fitness

### Termination Condition

In [ ]:
def check_end(population) -> bool:
    #TODO: check end of algorithm
    
    return ...

### Run Your Algorithms

In [ ]:
# The genetic algorithm
generation = 0
solution_is_found = False
fitness = []

population = generate_population()
population , population_fitness = population_fitness(population)

while not check_end(population):
    # TODO: this is a default setup which you maybe need to change.
    population = parent_selection(population)
    population = crossover(population)
    population = mutate(population)
    individuals_fitnesses , population_fitness = population_fitness(population)
    population = next_generation_selection(population)
    
    generation += 1
    avg_fit = population_fitness / population.__len__()
    fitness.append(avg_fit)


### See Your Progress Over Generations

In [ ]:
plt.bar(range(generation), fitness, color='skyblue')
plt.xlabel('Generation', fontsize=20)
plt.ylabel('Average Fitness', fontsize=20)
plt.title('Fitness over Generations', fontsize=20)
plt.show()

Test code

In [49]:
generation = 0
solution_is_found = False
fitness = []

population = generate_population()
population , population_fit = population_fitness(population)
print(population)
population = parent_selection(population)

0 4
1 1
2 6
3 4
1 5
1 7
0 3
0 4
0 6
1 2
3 4
3 6
4 6
5 6
5 7
6 7
0 2
1 3
2 5
4 5
1 6
1 7
0 2
1 2
1 4
3 4
1 5
1 6
1 7
0 4
1 1
1 2
3 5
3 7
4 5
4 6
5 6
5 7
1 0
1 1
1 2
3 5
4 6
4 7
6 7
0 2
1 2
3 4
4 6
1 5
1 7
1 0
1 2
1 3
1 5
1 6
2 3
1 4
5 6
1 7
1 0
1 1
2 5
1 3
1 4
1 6
1 7
0 1
0 2
1 3
4 6
1 5
1 7
1 0
1 2
3 5
3 6
4 7
5 6
0 1
1 4
1 5
2 3
4 5
5 7
1 6
0 3
1 2
2 4
3 5
5 6
1 7
1 0
1 6
1 2
1 3
1 4
5 7
1 0
1 1
1 2
3 4
4 6
5 7
1 0
1 3
1 5
2 7
3 5
4 6
0 1
0 5
2 3
2 4
1 6
1 7
0 3
1 2
3 4
1 5
1 6
1 7
0 5
0 6
1 2
2 4
1 3
5 6
1 7
0 2
1 2
3 4
4 6
4 7
1 5
6 7
{'2 8 5 7 6 3 1 4': 3, '1 3 2 4 5 8 7 6': 0, '6 3 8 1 4 5 7 2': 2, '8 5 6 1 2 4 7 3': 3, '2 8 1 3 6 5 4 7': 2, '7 1 3 6 2 8 4 5': 3, '8 5 6 2 3 7 1 4': 2, '6 3 2 1 5 7 8 4': 3, '6 8 5 1 4 2 7 3': 6, '5 4 7 1 8 2 6 3': 3, '6 8 7 5 1 3 2 4': 1, '4 5 7 6 2 1 8 3': 1, '8 2 1 5 3 7 6 4': 1, '4 7 5 3 1 6 2 8': 4, '1 3 5 7 8 4 6 2': 3, '8 5 7 3 6 1 4 2': 1, '6 7 3 2 5 1 8 4': 2, '5 3 4 8 7 2 6 1': 3, '2 5 6 1 4 7 8 3': 2, '4 1 2 6 5 3 7 8': 1}


In [46]:
population 

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

In [21]:

d = {repr([1,2,3]): 'value'}

In [31]:
type(list(d.keys())[0])

str